## 获取开盘价和收盘价满足一定条件的日期

In [31]:
import pandas as pd
import numpy as np
import tushare as ts


# 将日期作为索引，并修改收盘价为股票名
zgpa = ts.get_k_data('601318', start='2012-01-01', end='2014-12-31')
zgpa = zgpa.set_index('date')

# 获取收盘价比开盘价高3%的日期
condition1 = (zgpa['close']/zgpa['open']) - 1 >= 0.03
result1 = zgpa[condition1]
result1


,open,close,high,low,volume,code
date,,,,,,
2012-01-10,15.482,16.145,16.190,15.460,385120.0,601318
2012-01-17,15.918,16.826,16.870,15.771,312293.0,601318
2012-02-02,16.750,17.635,17.662,16.746,440393.0,601318
2012-04-05,16.274,16.799,16.906,16.078,288782.0,601318
2012-06-13,18.917,20.145,20.225,18.917,542936.0,601318
2012-07-19,19.752,20.553,20.804,19.690,353009.0,601318
2012-08-30,16.956,17.507,17.694,16.956,225262.0,601318
2012-09-03,17.238,17.829,17.958,17.234,191676.0,601318
2012-09-07,18.075,18.813,19.050,18.075,402051.0,601318


In [33]:
# 获取当日开盘价比上一日收盘价低%的日期
condition2 = (zgpa['open'] / zgpa['close'].shift(1)) - 1 <= -0.02
result2 = zgpa[condition2]
result2

,open,close,high,low,volume,code
date,,,,,,
2012-12-24,18.551,18.914,18.986,18.461,301313.0,601318
2013-03-04,20.208,19.166,20.297,18.905,805872.0,601318
2013-03-28,18.456,18.434,18.479,17.962,370700.0,601318
2014-12-09,28.872,27.447,31.056,26.998,4166545.0,601318
2014-12-11,26.836,26.290,27.493,26.193,1714298.0,601318


## 每月第一个交易日买入，每年最后一个交易日卖出的定投的收益

In [ ]:
# 如果不转换，数据类型是字符串类型
zgpa.index = pd.to_datetime(zgpa.index) 
# 获取每个月第一天的数据,'MS'中的'S'代表的是'Start'的意思，如果不加，显示出来的日期是月末日期
df_monthly = zgpa.resample('MS').first()
df_year = zgpa.resample('A').last()

# 假设每次按收盘价买1手，每年末全部卖出，计算总收益
cost = 0
hands = 0
for year in range(2012, 2015):
    # 每年购买花费的钱
    cost += df_monthly[str(year)]['close'].sum() * 100
    hands += len(df_monthly[str(year)]) *100
    
# 每年年末卖掉股票的总价值
sell = df_year['close'].sum() * hands

sell - cost